In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import requests
import datetime
import time
import warnings
import ta
warnings.filterwarnings('ignore')
from tensorflow import keras
from pylab import rcParams
from matplotlib import rc
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 16, 10

In [16]:
stocklist = si.tickers_sp500()
start_date = datetime.datetime.now() - datetime.timedelta(days=(365*3) + 30)
end_date = datetime.date.today()

In [17]:
def get_data(ticker,start_date,end_date):
    data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
    data['Ticker'] = ticker
    return data


def technical_analysis(sec):
    '''
    Contains all the indicators
    sec: security dataframe - should contain Close, Open, Low, High, and Volume of the desired security
    returns all the technical indicators
    
    '''
    #Momentum Indicators
    stoc_oscc_signal = ta.momentum.stoch_signal(high = sec.High, low = sec.Low,close = sec.Close, n=14, d_n=3, fillna=False)
    tsi = ta.momentum.tsi(close = sec.Close, r=25, s=13, fillna=False)
    awesome_oscillator = ta.momentum.AwesomeOscillatorIndicator(high = sec.High, low = sec.Low, s = 5, len = 34, fillna = False).ao()
    kama_indicator = ta.momentum.KAMAIndicator(close = sec.Close, n = 10, pow1 = 2, pow2 = 30, fillna = False).kama()
    roc_indicator = ta.momentum.ROCIndicator(close = sec.Close, n = 12, fillna = False).roc()
    rsi14 = ta.momentum.rsi(close = sec.Close, n=14, fillna=False)
    ultimate_osc = ta.momentum.UltimateOscillator(high = sec.High, low = sec.Low,close =  sec.Close, s = 7, m = 14, len = 28, ws = 4.0, wm = 2.0, wl = 1.0, fillna = False).uo()
    williamsR = ta.momentum.WilliamsRIndicator(high = sec.High, low = sec.Low,close =  sec.Close, lbp = 14, fillna = False).wr()
    
    #Volume Indicators
    adi = ta.volume.AccDistIndexIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, fillna = False).acc_dist_index()
    chaikin_money_flow = ta.volume.ChaikinMoneyFlowIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, n= 20, fillna = False).chaikin_money_flow()
    #ease_of_movement = ta.volume.EaseOfMovementIndicator(high = sec.High, low = sec.Low, volume = sec.Volume, n= 14, fillna = False).ease_of_movement()
    force_index_indicator = ta.volume.ForceIndexIndicator(close = sec.Close, volume = sec.Volume, n = 13, fillna = False).force_index()
    mfi = ta.volume.MFIIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume= sec.Volume, n = 14, fillna = False).money_flow_index()
    nvi = ta.volume.NegativeVolumeIndexIndicator(close = sec.Close, volume = sec.Volume, fillna= False).negative_volume_index()
    obv = ta.volume.OnBalanceVolumeIndicator(close = sec.Close, volume = sec.Volume, fillna = False).on_balance_volume()
    priceTrend = ta.volume.VolumePriceTrendIndicator(close = sec.Close, volume = sec.Volume, fillna = False).volume_price_trend()
    vwap = ta.volume.VolumeWeightedAveragePrice(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, n= 14, fillna= False).volume_weighted_average_price()
    
    #Volatility Indicators
    atr = ta.volatility.AverageTrueRange(high = sec.High, low = sec.Low, close = sec.Close, n= 14, fillna= False).average_true_range()
    
    #Trend Indicators
    adx = ta.trend.ADXIndicator(high = sec.High, low = sec.Low, close = sec.Close, n= 14, fillna = False).adx()
    aroon = ta.trend.AroonIndicator(close = sec.Close, n = 25, fillna = False).aroon_indicator()
    cci = ta.trend.CCIIndicator(high = sec.High, low = sec.Low, close = sec.Close, n = 20, c= 0.015, fillna= False).cci()
    dpo = ta.trend.DPOIndicator(close = sec.Close, n = 20, fillna = False).dpo()
    ema = ta.trend.EMAIndicator(close = sec.Close, n = 14, fillna = False).ema_indicator()
    kst = ta.trend.KSTIndicator(close= sec.Close, r1 = 10, r2 = 15, r3= 20, r4 = 30, n1= 10, n2= 10, n3 = 10, n4 = 15, nsig = 9, fillna= False).kst()
    macd = ta.trend.MACD(close = sec.Close, n_slow = 26, n_fast = 12, n_sign= 9, fillna = False).macd()
    
    tech = pd.DataFrame({'stoc_oscc_signal':stoc_oscc_signal,'tsi':tsi,'awesome_oscillator':awesome_oscillator,'kama_indicator':kama_indicator,
    'roc_indicator':roc_indicator,'rsi14':rsi14,'ultimate_osc':ultimate_osc,'williamsR':williamsR,
    'adi':adi,"chaikin_money_flow":chaikin_money_flow,'force_index_indicator':force_index_indicator,
    'mfi':mfi,'nvi':nvi,'obv':obv,'priceTrend':priceTrend,'vwap':vwap,'atr':atr,
    'adx':adx,'aroon':aroon,'cci':cci,'dpo':dpo,'ema':ema, 'kst':kst,'macd':macd})
    
    return tech


def merge_returns_and_technicals(price_df,technicals_df):
    ticker = price_df['Ticker'][0]
    price_df.drop(columns = ['Ticker'],inplace = True)
    returns = price_df.pct_change(1).rename(columns = {'Adj Close': 'Returns'})
    merged_df = technicals_df.join(returns['Returns'].shift(1)) #Lagged Returns
    merged_df['Ticker'] = ticker
    return merged_df


def master_data(security_list,start_date,end_date):
    final_df = pd.DataFrame()
    count = 0
    for security in security_list:
        try:
            sec = get_data(security,start_date = start_date,end_date = end_date)
            tech = technical_analysis(sec)
            merged = merge_returns_and_technicals(sec,tech)
            final_df = pd.concat([final_df,merged],axis = 0).dropna()
        except:
            pass
        count+=1
    return final_df

In [18]:
master = master_data(stocklist,start_date=start_date,end_date=end_date)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [21]:
master = master[['Ticker','Returns','stoc_oscc_signal', 'tsi', 'awesome_oscillator', 'kama_indicator',
       'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR', 'adi',
       'chaikin_money_flow', 'force_index_indicator',
       'mfi', 'nvi', 'obv', 'priceTrend', 'vwap', 'atr', 'adx', 'aroon', 'cci',
       'dpo', 'ema', 'kst', 'macd']]
X_cols = ['stoc_oscc_signal', 'tsi', 'awesome_oscillator',
       'kama_indicator', 'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR',
       'adi', 'chaikin_money_flow',
       'force_index_indicator', 'mfi', 'nvi', 'obv', 'priceTrend', 'vwap',
       'atr', 'adx', 'cci', 'dpo', 'ema', 'kst', 'macd']
for i in X_cols:
    master[i] = (master[i] - master[i].rolling(window=100).mean())/(master[i].rolling(window=100).std())
master.dropna(inplace = True)
y = np.array((master['Returns'] > 0)*1)
X = master[['stoc_oscc_signal', 'tsi', 'awesome_oscillator',
       'kama_indicator', 'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR',
       'adi', 'chaikin_money_flow',
       'force_index_indicator', 'mfi', 'nvi', 'obv', 'priceTrend', 'vwap',
       'atr', 'adx', 'cci', 'dpo', 'ema', 'kst', 'macd']]
X = (X - X.rolling(100).mean())/(X.rolling(100).std())
df_std = X.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(X.keys(), rotation=90)

In [43]:
X_total = X.iloc[100:]
y_total = pd.DataFrame(y)[100:]
train_size = int(len(X_total) * 0.8)
test_size = len(X_total) - train_size
train_X, test_X = X_total.iloc[0:train_size], X_total.iloc[train_size:len(X_total)]
train_y, test_y = y_total.iloc[0:train_size], y_total.iloc[train_size:len(X_total)]
print(len(train_X), len(test_X))
print(len(train_y), len(test_y))

In [3]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [50]:
time_steps = 10
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train_X, train_y, time_steps)
X_test, y_test = create_dataset(test_X, test_y, time_steps)
print(X_train.shape, y_train.shape)
model = keras.Sequential()
model.add(keras.layers.LSTM(
  units=128,
  input_shape=(X_train.shape[1], X_train.shape[2])
))
model.add(keras.layers.Dense(units=1))
model.compile(
  loss='mean_squared_error',
  optimizer=keras.optimizers.Adam(0.001)
)
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

(251236, 10, 23) (251236, 1)


Epoch 1/30
14132/14132 [==============================] - 167s 12ms/step - loss: 0.0491 - val_loss: 0.0262
Epoch 2/30
14132/14132 [==============================] - 165s 12ms/step - loss: 0.0252 - val_loss: 0.0210
Epoch 3/30
14132/14132 [==============================] - 170s 12ms/step - loss: 0.0208 - val_loss: 0.0190
Epoch 4/30
14132/14132 [==============================] - 163s 12ms/step - loss: 0.0183 - val_loss: 0.0180
Epoch 5/30
14132/14132 [==============================] - 166s 12ms/step - loss: 0.0166 - val_loss: 0.0171
Epoch 6/30
14132/14132 [==============================] - 168s 12ms/step - loss: 0.0153 - val_loss: 0.0170
Epoch 7/30
14132/14132 [==============================] - 168s 12ms/step - loss: 0.0143 - val_loss: 0.0171
Epoch 8/30
14132/14132 [==============================] - 168s 12ms/step - loss: 0.0134 - val_loss: 0.0173
Epoch 9/30
14132/14132 [==============================] - 169s 12ms/step - loss: 0.0128 - val_loss: 0.0179
Epoch 10/30
14132/14132 [============

In [53]:
y_pred = model.predict(X_test)

In [69]:
results = pd.DataFrame()
results['Pred'] = pd.DataFrame(y_pred)[0]
results['Actual'] = pd.DataFrame(y_test)[0]

In [71]:
results['Pred'] = np.where(results['Pred'] > 0.5,1,0)

In [75]:
np.where(results['Pred'] == results['Actual'],1,0).sum()/len(results)

0.9805579440145218